In [1]:
import sqlite3, os, time
import pandas as pd
import numpy as np

In [2]:
path = os.getcwd()+'/'

## preprocess data

In [ ]:
data = pd.read_csv(path+'card transactions.csv')
#data=pd.read_excel(path+"card transactions.xlsx")
data.rename(columns={"Merch Description": "Merch_Description", 
                      "Merchant State": "Merchant_State",
                      'Merchant Zip': 'Merchant_Zip',
                      ' Amount ':'Amount'
                    },
           inplace = True)
dates = data['Date'].unique()
data['date_index'] = data['Date'].apply(lambda x: int(np.where(dates==x)[0]))

In [9]:
data = data.query('Transtype == "P"')

In [10]:
data.head()

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,Unnamed: 10,date_index
0,1,5142190439,01/01/2010,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,$3.62,0,NaN,0
1,2,5142183973,01/01/2010,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,$31.42,0,NaN,0
2,3,5142131721,01/01/2010,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,$178.49,0,NaN,0
3,4,5142148452,01/01/2010,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,$3.62,0,NaN,0
4,5,5142190439,01/01/2010,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,$3.62,0,NaN,0


In [ ]:
type(data.loc[1,"Amount"])
for index,cont in data.iterrows():
    a=data.loc[index,"Amount"]
    a=a.replace("$", "")
    a=a.replace(",", "")
    a=float(a)
    data.loc[index,"Amount"]=a
data.to_excel("changed fraud origional data.xlsx")

In [ ]:
#data = pd.read_excel(path+'changed fraud origional data.xlsx')

### exchanging peso into dollar

In [15]:
data['Amount'].max()

3102045.53

In [16]:
data.query('Amount == 3102045.53')

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,Unnamed: 10,date_index
52593,52594,5142189135,13/07/2010,NaN,INTERMEXICO,NaN,NaN,P,3102045.53,0,NaN,193


In [17]:
data.loc[52593,'Amount'] = data.loc[52593,'Amount']*0.079

In [18]:
data.loc[52593,:]

Recordnum                  52594
Cardnum               5142189135
Date                  13/07/2010
Merchantnum                  NaN
Merch_Description    INTERMEXICO
Merchant_State               NaN
Merchant_Zip                 NaN
Transtype                      P
Amount                    245062
Fraud                          0
Unnamed: 10                  NaN
date_index                   193
Name: 52593, dtype: object

In [19]:
data['Amount'].max()

245061.59686999998

### merchnum first occurence 

must be done before inserting missing values

In [20]:
global_list = []
def in_list(x):
    global global_list
    if x not in global_list:
        global_list.append(x)
        return False
    else:
        return True   

In [21]:
data['new_merch'] = data['Merchantnum'].apply(lambda x: 0 if in_list(x) == True else 1)

### heuristic time series difference

#### Monday=0, Sunday=6
#### January=1, December=12

In [22]:
data['weekday'] = pd.to_datetime(data['Date']).dt.dayofweek
data['month'] = pd.to_datetime(data['Date']).dt.month

In [23]:
avg_weekday = {}
avg_month = {}
for i in range(7):
    avg_weekday[i]=np.mean(data.query('weekday == @i')['Amount'])
for i in range(1,13):
    avg_month[i]=np.mean(data.query('month == @i')['Amount'])

In [24]:
avg_weekday

{0: 403.200321040697,
 1: 410.3607603518444,
 2: 419.96639165670257,
 3: 386.39910384068276,
 4: 375.42914775160597,
 5: 381.39514858639683,
 6: 393.39845331139446}

In [25]:
avg_month

{1: 376.733940146367,
 2: 366.36978267211197,
 3: 364.9744956709957,
 4: 402.8143892025892,
 5: 384.21363636363634,
 6: 399.46352079341824,
 7: 423.6491664187804,
 8: 386.08102218620195,
 9: 438.49741791993216,
 10: 422.3481885931559,
 11: 457.01811054184134,
 12: 360.0368212306795}

In [26]:
data.head()

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,Unnamed: 10,date_index,new_merch,weekday,month
0,1,5142190439,01/01/2010,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,NaN,0,1,4,1
1,2,5142183973,01/01/2010,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,NaN,0,1,4,1
2,3,5142131721,01/01/2010,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,NaN,0,1,4,1
3,4,5142148452,01/01/2010,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,NaN,0,0,4,1
4,5,5142190439,01/01/2010,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,NaN,0,0,4,1


In [27]:
week_diff = []
month_diff = []
for i,row in data.iterrows():
    week_diff.append(row['Amount']-avg_weekday[row['weekday']])
    month_diff.append(row['Amount']-avg_month[row['month']])

In [28]:
data['Amount_week_diff'] = week_diff
data['Amount_month_diff'] = month_diff

### missing value processing

In [29]:
summary_table = data.describe(include='all').transpose().iloc[:,:2]
summary_table['populated_rate'] = summary_table['count']/len(data)
summary_table

,count,unique,populated_rate
Recordnum,96353,NaN,1
Cardnum,96353,NaN,1
Date,96353,365,1
Merchantnum,93154,13089,0.966799
Merch_Description,96353,12966,1
Merchant_State,95332,59,0.989404
Merchant_Zip,92052,NaN,0.955362
Transtype,96353,1,1
Amount,96353,NaN,1
Fraud,96353,NaN,1


Since we assign each na different value to prevent inflating the burst in `Merchantnum`, we need isna identifier to prevent inflating different `Merchant_State` and `Merchant_Zip`

In [30]:
data['Merchantnum_isna'] = data['Merchantnum'].isna().astype('int')
data['Merchant_State_isna'] = data['Merchant_State'].isna().astype('int')
data['Merchant_Zip_isna'] = data['Merchant_Zip'].isna().astype('int')

In [31]:
1-np.mean(data['Merchant_Zip_isna'])

0.95536205411352

In [32]:
data['for_na'] = np.array(['n_'+str(d) for d in data['Recordnum']])

In [33]:
data['Merchantnum'].fillna(data['for_na'],inplace=True)
data['Merchant_State'].fillna(data['for_na'],inplace=True)
data['Merchant_Zip'].fillna(data['for_na'],inplace=True)

In [34]:
data.query('Merchant_Zip.isna()')

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,...,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na


In [35]:
data.tail(10)

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,...,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na
96698,96699,5142187448,31/12/2010,5725000466504,CDW*GOVERNMENT INC,IL,60061,P,2231.08,0,...,364,0,4,12,1855.650852,1871.043179,0,0,0,n_96699
96699,96700,5142160778,31/12/2010,08-0616075333,COMPUTER STORE OF CORVLS,OR,97330,P,243.15,1,...,364,0,4,12,-132.279148,-116.886821,0,0,0,n_96700
96700,96701,5142226979,31/12/2010,602608969534,FISHER SCI ATL,GA,30091,P,32.04,0,...,364,0,4,12,-343.389148,-327.996821,0,0,0,n_96701
96701,96702,5142196337,31/12/2010,467619770330,STAY ONLINE,NC,27560,P,147.35,0,...,364,0,4,12,-228.079148,-212.686821,0,0,0,n_96702
96702,96703,5142221253,31/12/2010,5725000466504,CDW*GOVERNMENT INC,IL,60061,P,425.00,0,...,364,0,4,12,49.570852,64.963179,0,0,0,n_96703
96703,96704,5142276053,31/12/2010,3500000006160,BEST BUY 00001610,KY,41042,P,84.79,0,...,364,1,4,12,-290.639148,-275.246821,0,0,0,n_96704
96704,96705,5142225701,31/12/2010,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248,P,118.75,0,...,364,0,4,12,-256.679148,-241.286821,0,0,0,n_96705
96705,96706,5142226486,31/12/2010,4503057341100,"TECH PAC, INC",OH,45150,P,363.56,0,...,364,0,4,12,-11.869148,3.523179,0,0,0,n_96706
96706,96707,5142244619,31/12/2010,8834000695412,BUY.COM,CA,92656,P,2202.03,0,...,364,0,4,12,1826.600852,1841.993179,0,0,0,n_96707
96707,96708,5142243247,31/12/2010,9108347680006,STAPLES NATIONAL #471,NJ,7606,P,554.64,0,...,364,0,4,12,179.210852,194.603179,0,0,0,n_96708


## Load data into sqlite and create variables

In [36]:
conn = sqlite3.connect(path+'fraud.sqlite')
#drop tables
conn.cursor().execute('drop table if exists card_transactions;')
data.to_sql(name='card_transactions', con = conn, index = False, chunksize = None)

/Users/yufeiwang/anaconda3/envs/py27/lib/python2.7/site-packages/pandas/core/generic.py:1534: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [37]:
def pysql(sqlcode):
    return pd.read_sql_query(sqlcode, conn)

In [38]:
sql_code = '''
select * from card_transactions limit 0,3;
'''

In [39]:
pysql(sql_code)

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,...,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na
0,1,5142190439,01/01/2010,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,...,0,1,4,1,-371.809148,-373.11394,0,0,0,n_1
1,2,5142183973,01/01/2010,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,...,0,1,4,1,-344.009148,-345.31394,0,0,0,n_2
2,3,5142131721,01/01/2010,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,...,0,1,4,1,-196.939148,-198.24394,0,0,0,n_3


In [40]:
pysql('select count(distinct date_index) as num_unique_dates from card_transactions;')

,num_unique_dates
0,365


In [41]:
pysql('select count(*) as length from card_transactions;')

,length
0,96353


In [42]:
sql_code = '''
SELECT t1.Recordnum,

--- 0. basic statistics
-- 0.0. avg amount 30, 15, 7, 3, 1

AVG(t2.Amount) As cardnum_amount_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As cardnum_amount_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As cardnum_amount_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As cardnum_amount_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As cardnum_amount_avg_1,

-- 0.1. max amount 30, 15, 7, 3, 1

MAX(t2.Amount) As cardnum_amount_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As cardnum_amount_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As cardnum_amount_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As cardnum_amount_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As cardnum_amount_max_1,

-- 0.2. sum amount 30, 15, 7, 3, 1

SUM(t2.Amount) As cardnum_amount_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As cardnum_amount_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As cardnum_amount_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As cardnum_amount_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As cardnum_amount_sum_1,

-- 0.3. avg amount week_diff 30, 15, 7, 3, 1

AVG(t2.Amount_week_diff) As cardnum_amount_week_diff_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_avg_1,

-- 0.4. max amount week_diff 30, 15, 7, 3, 1

MAX(t2.Amount_week_diff) As cardnum_amount_week_diff_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_max_1,

-- 0.5. sum amount week_diff 30, 15, 7, 3, 1

SUM(t2.Amount_week_diff) As cardnum_amount_week_diff_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_sum_1,

-- 0.6. avg amount month_diff 30, 15, 7, 3, 1

AVG(t2.Amount_month_diff) As cardnum_amount_month_diff_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_avg_1,

-- 0.7. max amount month_diff 30, 15, 7, 3, 1

MAX(t2.Amount_month_diff) As cardnum_amount_month_diff_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_max_1,

-- 0.8. sum amount month_diff 30, 15, 7, 3, 1

SUM(t2.Amount_month_diff) As cardnum_amount_month_diff_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_sum_1,

--- 1. burst
-- 1.0. counting the same card number in the past 30,15,7,3,1 days

COUNT(*) AS cardnum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN 1 ELSE NULL END) AS cardnum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN 1 ELSE NULL END) AS cardnum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN 1 ELSE NULL END) AS cardnum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN 1 ELSE NULL END) AS cardnum_1,

--- 2. new merchant
-- 2.0. counting new merchants given card number in the past 30,15,7,3,1 days

SUM(t2.new_merch) AS newmerch_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.new_merch ELSE NULL END) AS newmerch_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.new_merch ELSE NULL END) AS newmerch_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.new_merch ELSE NULL END) AS newmerch_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.new_merch ELSE NULL END) AS newmerch_1,

--- 3. Merchantnum
-- 3.0. how many different Merchantnum for a given Cardnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchantnum) AS cardnum_merchantnum_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchantnum ELSE NULL END) 
AS cardnum_merchantnum_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchantnum ELSE NULL END) 
AS cardnum_merchantnum_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchantnum ELSE NULL END) 
AS cardnum_merchantnum_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchantnum ELSE NULL END) 
AS cardnum_merchantnum_1,

-- 3.1. how many originally na Merchantnum for a given Cardnum in 30,15,7,3,1 days 

SUM(t2.Merchantnum_isna) AS cardnum_merchantnum_isna_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchantnum_isna ELSE NULL END) 
AS cardnum_merchantnum_isna_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchantnum_isna ELSE NULL END) 
AS cardnum_merchantnum_isna_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchantnum_isna ELSE NULL END) 
AS cardnum_merchantnum_isna_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchantnum_isna ELSE NULL END) 
AS cardnum_merchantnum_isna_1,

--- 4. zip
-- 4.0. how many different zip for a given Cardnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchant_Zip) AS cardnum_zip_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_Zip ELSE NULL END) 
AS cardnum_zip_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_Zip ELSE NULL END) 
AS cardnum_zip_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_Zip ELSE NULL END) 
AS cardnum_zip_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_Zip ELSE NULL END) 
AS cardnum_zip_1,

-- 4.1. how many originally na zip for a given Cardnum in 30,15,7,3,1 days 

SUM(t2.Merchant_Zip_isna) AS cardnum_zip_isna_30,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS cardnum_zip_isna_15,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS cardnum_zip_isna_7,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS cardnum_zip_isna_3,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS cardnum_zip_isna_1,

--- 5. states
-- 5.0. how many different states for a given Cardnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchant_State) AS cardnum_state_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_State ELSE NULL END) 
AS cardnum_state_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_State ELSE NULL END) 
AS cardnum_state_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_State ELSE NULL END) 
AS cardnum_state_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_State ELSE NULL END) 
AS cardnum_state_1,

-- 5.1. how many originally na states for a given Cardnum in 30,15,7,3,1 days 

SUM(t2.Merchant_State_isna) AS cardnum_state_isna_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_State_isna ELSE NULL END) 
AS cardnum_state_isna_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_State_isna ELSE NULL END) 
AS cardnum_state_isna_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_State_isna ELSE NULL END) 
AS cardnum_state_isna_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_State_isna ELSE NULL END) 
AS cardnum_state_isna_1

-- A self join is applied, in this case either left or inner join is fine 
FROM card_transactions t1 LEFT JOIN card_transactions t2

-- Matching condition 1: the same Cardnum
ON t1.Cardnum = t2.Cardnum

-- Matching condition 2: within the (largest) time interval
AND t1.date_index - t2.date_index BETWEEN 0 AND 29

-- Matching condition 3: If a Cardnum has multiple records on the same day,
--                       only match the records before it (presumably they are in chronological order)
AND t1.Recordnum >= t2.Recordnum

-- Now each record has matched all the corresponding records in the specified time window,
-- we can group by the record id to calculate the four aggregated metrics
GROUP BY t1.Recordnum
'''

In [43]:
start = time.time()
cardnum = pysql(sql_code)
end = time.time()
print ('time elipsed: {:.2f}'.format(end-start))

time elipsed: 35.76


In [44]:
cardnum.tail()

,Recordnum,cardnum_amount_avg_30,cardnum_amount_avg_15,cardnum_amount_avg_7,cardnum_amount_avg_3,cardnum_amount_avg_1,cardnum_amount_max_30,cardnum_amount_max_15,cardnum_amount_max_7,cardnum_amount_max_3,...,cardnum_state_30,cardnum_state_15,cardnum_state_7,cardnum_state_3,cardnum_state_1,cardnum_state_isna_30,cardnum_state_isna_15,cardnum_state_isna_7,cardnum_state_isna_3,cardnum_state_isna_1
96348,96704,84.790000,84.790000,84.790000,84.7900,84.790,84.79,84.79,84.79,84.79,...,1,1,1,1,1,0,0,0,0,0
96349,96705,628.382500,622.246154,628.050000,628.0500,401.875,1705.60,1705.60,1478.00,1478.00,...,4,4,3,3,2,0,0,0,0,0
96350,96706,442.190769,429.688333,429.688333,386.6375,363.560,1065.73,1065.73,1065.73,1065.73,...,7,3,3,2,1,0,0,0,0,0
96351,96707,865.614000,1261.490000,2202.030000,2202.0300,2202.030,2202.03,2202.03,2202.03,2202.03,...,3,3,1,1,1,0,0,0,0,0
96352,96708,322.175667,1760.616000,1076.251667,796.1040,554.640,6964.90,6964.90,3142.52,3142.52,...,9,6,3,2,1,0,0,0,0,0


In [45]:
cardnum['cardnum_amount_max_3'].tail(1)

96352    3142.52
Name: cardnum_amount_max_3, dtype: float64

In [46]:
cardnum['cardnum_amount_avg_3'].tail(1)

96352    796.104
Name: cardnum_amount_avg_3, dtype: float64

In [47]:
cardnum['cardnum_3'].tail(1)

96352    5
Name: cardnum_3, dtype: int64

In [48]:
last = data.query('date_index >= 362')

In [49]:
last.Cardnum.iloc[-1]

5142243247

In [50]:
last.query('Cardnum==5142243247')

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,...,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na
96205,96206,5142243247,29/12/2010,61563,A DAIGGER & CO INC,IL,60069,P,116.20,0,...,362,0,2,12,-303.766392,-243.836821,0,0,0,n_96206
96251,96252,5142243247,29/12/2010,9108347680006,STAPLES NATIONAL #471,NJ,7606,P,76.80,0,...,362,0,2,12,-343.166392,-283.236821,0,0,0,n_96252
96376,96377,5142243247,29/12/2010,9108347680000,STAPLES NATIONAL #471,NJ,7606,P,3142.52,0,...,362,0,2,12,2722.553608,2782.483179,0,0,0,n_96377
96397,96398,5142243247,29/12/2010,9108347680006,STAPLES NATIONAL #471,NJ,7606,P,90.36,0,...,362,0,2,12,-329.606392,-269.676821,0,0,0,n_96398
96707,96708,5142243247,31/12/2010,9108347680006,STAPLES NATIONAL #471,NJ,7606,P,554.64,0,...,364,0,4,12,179.210852,194.603179,0,0,0,n_96708


In [51]:
np.mean(last.query('Cardnum==5142243247')['Amount'])

796.104

In [52]:
sql_code_2 = '''
SELECT t1.Recordnum,

--- 0. basic statistics
-- 0.0. avg amount 30, 15, 7, 3, 1

AVG(t2.Amount) As merchantnum_amount_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As merchantnum_amount_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As merchantnum_amount_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As merchantnum_amount_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As merchantnum_amount_avg_1,

-- 0.1. max amount 30, 15, 7, 3, 1

MAX(t2.Amount) As merchantnum_amount_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As merchantnum_amount_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As merchantnum_amount_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As merchantnum_amount_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As merchantnum_amount_max_1,

-- 0.2. sum amount 30, 15, 7, 3, 1

SUM(t2.Amount) As merchantnum_amount_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As merchantnum_amount_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As merchantnum_amount_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As merchantnum_amount_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As merchantnum_amount_sum_1,

-- 0.3. avg amount week_diff 30, 15, 7, 3, 1

AVG(t2.Amount_week_diff) As merchantnum_amount_week_diff_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_avg_1,

-- 0.4. max amount week_diff 30, 15, 7, 3, 1

MAX(t2.Amount_week_diff) As merchantnum_amount_week_diff_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_max_1,

-- 0.5. sum amount week_diff 30, 15, 7, 3, 1

SUM(t2.Amount_week_diff) As merchantnum_amount_week_diff_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_sum_1,

-- 0.6. avg amount month_diff 30, 15, 7, 3, 1

AVG(t2.Amount_month_diff) As merchantnum_amount_month_diff_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_avg_1,

-- 0.7. max amount month_diff 30, 15, 7, 3, 1

MAX(t2.Amount_month_diff) As merchantnum_amount_month_diff_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_max_1,

-- 0.8. sum amount month_diff 30, 15, 7, 3, 1

SUM(t2.Amount_month_diff) As merchantnum_amount_month_diff_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_sum_1,

--- 1. burst
-- 1.0. counting the same card number in the past 30,15,7,3,1 days

COUNT(*) AS merchantnum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN 1 ELSE NULL END) AS merchantnum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN 1 ELSE NULL END) AS merchantnum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN 1 ELSE NULL END) AS merchantnum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN 1 ELSE NULL END) AS merchantnum_1,

--- 2. Cardnum
-- 2.0. how many different Cardnum for a given Merchantnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Cardnum) AS merchantnum_cardnum_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Cardnum ELSE NULL END) 
AS merchantnum_cardnum_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Cardnum ELSE NULL END) 
AS merchantnum_cardnum_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Cardnum ELSE NULL END) 
AS merchantnum_cardnum_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Cardnum ELSE NULL END) 
AS merchantnum_cardnum_1,


--- 3. Zip
-- 3.0. how many different zip for a given Merchantnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchant_Zip) AS merchantnum_zip_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_Zip ELSE NULL END) 
AS merchantnum_zip_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_Zip ELSE NULL END) 
AS merchantnum_zip_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_Zip ELSE NULL END) 
AS merchantnum_zip_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_Zip ELSE NULL END) 
AS merchantnum_zip_1,

-- 3.1. how many originally na zip for a given Merchantnum in 30,15,7,3,1 days 

SUM(t2.Merchant_Zip_isna) AS merchantnum_zip_isna_30,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS merchantnum_zip_isna_15,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS merchantnum_zip_isna_7,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS merchantnum_zip_isna_3,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS merchantnum_zip_isna_1,

--- 4. states
-- 4.0. how many different states for a given Merchantnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchant_State) AS merchantnum_state_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_State ELSE NULL END) 
AS merchantnum_state_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_State ELSE NULL END) 
AS merchantnum_state_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_State ELSE NULL END) 
AS merchantnum_state_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_State ELSE NULL END) 
AS merchantnum_state_1,

-- 5.1. how many originally na states for a given Merchantnum in 30,15,7,3,1 days 

SUM(t2.Merchant_State_isna) AS merchantnum_state_isna_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_State_isna ELSE NULL END) 
AS merchantnum_state_isna_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_State_isna ELSE NULL END) 
AS merchantnum_state_isna_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_State_isna ELSE NULL END) 
AS merchantnum_state_isna_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_State_isna ELSE NULL END) 
AS merchantnum_state_isna_1



-- A self join is applied, in this case either left or inner join is fine 
FROM card_transactions t1 LEFT JOIN card_transactions t2

-- Matching condition 1: the same Merchantnum
ON t1.Merchantnum = t2.Merchantnum

-- Matching condition 2: within the (largest) time interval
AND t1.date_index - t2.date_index BETWEEN 0 AND 29

-- Matching condition 3: If a Merchantnum has multiple records on the same day,
--                       only match the records before it (presumably they are in chronological order)
AND t1.Recordnum >= t2.Recordnum

-- Now each record has matched all the corresponding records in the specified time window,
-- we can group by the record id to calculate the four aggregated metrics
GROUP BY t1.Recordnum
'''

In [53]:
start = time.time()
merchantnum = pysql(sql_code_2)
end = time.time()
print ('time elipsed: {:.2f}'.format(end-start))

time elipsed: 132.85


In [54]:
merchantnum.tail()

,Recordnum,merchantnum_amount_avg_30,merchantnum_amount_avg_15,merchantnum_amount_avg_7,merchantnum_amount_avg_3,merchantnum_amount_avg_1,merchantnum_amount_max_30,merchantnum_amount_max_15,merchantnum_amount_max_7,merchantnum_amount_max_3,...,merchantnum_state_30,merchantnum_state_15,merchantnum_state_7,merchantnum_state_3,merchantnum_state_1,merchantnum_state_isna_30,merchantnum_state_isna_15,merchantnum_state_isna_7,merchantnum_state_isna_3,merchantnum_state_isna_1
96348,96704,84.790000,84.790000,84.7900,84.7900,84.7900,84.79,84.79,84.79,84.79,...,1,1,1,1,1,0,0,0,0,0
96349,96705,391.494286,324.080000,118.7500,118.7500,118.7500,1050.12,763.53,118.75,118.75,...,1,1,1,1,1,0,0,0,0,0
96350,96706,1191.567692,954.861429,1073.7050,1445.2225,1445.2225,2494.40,2487.98,2487.98,2487.98,...,1,1,1,1,1,0,0,0,0,0
96351,96707,729.972128,716.041538,1430.3075,1430.3075,2202.0300,5013.39,2349.95,2312.55,2312.55,...,1,1,1,1,1,0,0,0,0,0
96352,96708,106.438462,146.673333,240.6000,240.6000,554.6400,554.64,554.64,554.64,554.64,...,1,1,1,1,1,0,0,0,0,0


## Data cleaning

before correcting missing value encodings

In [55]:
cardnum[['cardnum_merchantnum_30','cardnum_zip_30','cardnum_state_30']].describe()

,cardnum_merchantnum_30,cardnum_zip_30,cardnum_state_30
count,96353.000000,96353.000000,96353.000000
mean,9.017010,8.670555,5.537056
std,8.451985,8.315366,4.076208
min,1.000000,1.000000,1.000000
25%,4.000000,3.000000,3.000000
50%,7.000000,6.000000,5.000000
75%,12.000000,11.000000,7.000000
max,86.000000,114.000000,31.000000


In [56]:
lags = ['1','3','7','15','30']
card_missing_reset = ['merchantnum','zip','state']
merchant_missing_reset = ['zip','state']
for l in lags:
    for c in card_missing_reset:
        cardnum['cardnum_{}_{}'.format(c,l)] = (cardnum['cardnum_{}_{}'.format(c,l)]-
                                               cardnum['cardnum_{}_isna_{}'.format(c,l)])
    for m in merchant_missing_reset:
        merchantnum['merchantnum_{}_{}'.format(m,l)] = (merchantnum['merchantnum_{}_{}'.format(m,l)]-
                                               merchantnum['merchantnum_{}_isna_{}'.format(m,l)])

after correcting missing value encodings

In [57]:
cardnum[['cardnum_merchantnum_30','cardnum_zip_30','cardnum_state_30']].describe()

,cardnum_merchantnum_30,cardnum_zip_30,cardnum_state_30
count,96353.000000,96353.000000,96353.000000
mean,8.538561,7.934346,5.399707
std,7.914273,7.331482,3.910024
min,0.000000,0.000000,0.000000
25%,3.000000,3.000000,2.000000
50%,6.000000,6.000000,5.000000
75%,11.000000,10.000000,7.000000
max,80.000000,73.000000,30.000000


## Data Aggregating

In [58]:
data.head()

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,...,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na
0,1,5142190439,01/01/2010,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,0,1,4,1,-371.809148,-373.11394,0,0,0,n_1
1,2,5142183973,01/01/2010,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0,...,0,1,4,1,-344.009148,-345.31394,0,0,0,n_2
2,3,5142131721,01/01/2010,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,...,0,1,4,1,-196.939148,-198.24394,0,0,0,n_3
3,4,5142148452,01/01/2010,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,...,0,0,4,1,-371.809148,-373.11394,0,0,0,n_4
4,5,5142190439,01/01/2010,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,...,0,0,4,1,-371.809148,-373.11394,0,0,0,n_5


In [59]:
output = pd.concat([cardnum.set_index('Recordnum'),
                    merchantnum.set_index('Recordnum'),
                    data[['Recordnum','Fraud','date_index']].set_index('Recordnum')],axis=1)

In [60]:
output.shape

(96353, 162)

In [61]:
dates[303]

'31/10/2010'

In [62]:
train = output.query('date_index <= 303')
train = train.iloc[:,:-1]

In [63]:
out_of_date = output.query('date_index > 303')
out_of_date = out_of_date.iloc[:,:-1]

In [64]:
train.shape

(83767, 161)

In [65]:
out_of_date.shape

(12586, 161)

In [66]:
train.tail()

,cardnum_amount_avg_30,cardnum_amount_avg_15,cardnum_amount_avg_7,cardnum_amount_avg_3,cardnum_amount_avg_1,cardnum_amount_max_30,cardnum_amount_max_15,cardnum_amount_max_7,cardnum_amount_max_3,cardnum_amount_max_1,...,merchantnum_state_15,merchantnum_state_7,merchantnum_state_3,merchantnum_state_1,merchantnum_state_isna_30,merchantnum_state_isna_15,merchantnum_state_isna_7,merchantnum_state_isna_3,merchantnum_state_isna_1,Fraud
Recordnum,,,,,,,,,,,,,,,,,,,,,
84091,344.3800,299.770000,299.770,299.77,299.77,929.00,299.77,299.77,299.77,299.77,...,1,1,1,1,0,0,0,0,0,0
84092,394.6600,394.660000,394.660,394.66,394.66,609.34,609.34,609.34,609.34,609.34,...,1,1,1,1,0,0,0,0,0,0
84093,221.5875,218.033333,127.475,235.00,235.00,399.15,399.15,235.00,235.00,235.00,...,1,1,1,1,0,0,0,0,0,0
84094,600.0000,600.000000,600.000,600.00,600.00,600.00,600.00,600.00,600.00,600.00,...,1,1,1,1,0,0,0,0,0,0
84095,197.6600,30.000000,30.000,30.00,30.00,365.32,30.00,30.00,30.00,30.00,...,1,1,1,1,0,0,0,0,0,0


In [ ]:
import random
random.seed(1)
testindex=random.sample(train.index,int(max(train.index)*4/10))
test=train.loc[testindex,:]
train=train.drop(testindex)
test=test.reset_index(drop=True)
train=train.reset_index(drop=True)
max(test.index)+max(train.index)

In [90]:
train.to_csv(path+'train.csv',index=False)
test.to_csv(path+"test.csv",index=False)
out_of_date.to_csv(path+'out_of_date.csv',index=False)

In [85]:
for i in train.columns:
    print i

cardnum_amount_avg_30
cardnum_amount_avg_15
cardnum_amount_avg_7
cardnum_amount_avg_3
cardnum_amount_avg_1
cardnum_amount_max_30
cardnum_amount_max_15
cardnum_amount_max_7
cardnum_amount_max_3
cardnum_amount_max_1
cardnum_amount_sum_30
cardnum_amount_sum_15
cardnum_amount_sum_7
cardnum_amount_sum_3
cardnum_amount_sum_1
cardnum_amount_week_diff_avg_30
cardnum_amount_week_diff_avg_15
cardnum_amount_week_diff_avg_7
cardnum_amount_week_diff_avg_3
cardnum_amount_week_diff_avg_1
cardnum_amount_week_diff_max_30
cardnum_amount_week_diff_max_15
cardnum_amount_week_diff_max_7
cardnum_amount_week_diff_max_3
cardnum_amount_week_diff_max_1
cardnum_amount_week_diff_sum_30
cardnum_amount_week_diff_sum_15
cardnum_amount_week_diff_sum_7
cardnum_amount_week_diff_sum_3
cardnum_amount_week_diff_sum_1
cardnum_amount_month_diff_avg_30
cardnum_amount_month_diff_avg_15
cardnum_amount_month_diff_avg_7
cardnum_amount_month_diff_avg_3
cardnum_amount_month_diff_avg_1
cardnum_amount_month_diff_max_30
cardnum_amoun

83765